In [1]:
import io
import pandas as pd
from google.colab import files

searches = files.upload()
enquiries = files.upload()
listings = files.upload()
users = files.upload()


Saving 99.co Data Analyst Challenge - searches.tsv to 99.co Data Analyst Challenge - searches.tsv


Saving 99.co Data Analyst Challenge - enquiries.tsv to 99.co Data Analyst Challenge - enquiries.tsv


Saving 99.co Data Analyst Challenge - listings.tsv to 99.co Data Analyst Challenge - listings.tsv


Saving 99.co Data Analyst Challenge - users.tsv to 99.co Data Analyst Challenge - users.tsv


## **Data Preprocessing & Feature Engineering**

During this stage, there are some considerations below.

1. There are 13% of these searches that 
users did enquiries. When users enquiries, it indicates that they are interested to the listings and would like to receive notifications on the update of the listings. Details in Excel sheets '-pivot search & enq'
2. We are going to segment these searches by sessions based on userid, platform and time difference.



In [20]:
!pip install kmodes

ERROR: Operation cancelled by user


In [113]:
from kmodes.kmodes import KModes
searches_df = pd.read_csv(io.BytesIO(searches['99.co Data Analyst Challenge - searches.tsv']), sep='\t')
searches_df['search_time'] = pd.to_datetime(searches_df['search_time'])
searches_df['session_duration'] = searches_df['search_time'] - searches_df['search_time'].shift(periods=1, axis=0)

searches_df['search_date'] = pd.to_datetime(searches_df['search_time']).dt.strftime('%Y-%m-%d')

del searches_df['search_time']
searches_df.head()

,search_id,listing_type,property_type,search_location,search_price_min,search_price_max,platform,user_id,session_duration,search_date
0,1,rent,condo,"dtdistrict15,dtdistrict16,dtdistrict18",2000.0,4000.0,web,u1386,NaT,2018-07-01
1,2,rent,condo,"dtdistrict15,dtdistrict16,dtdistrict18",2000.0,4000.0,web,u1386,0 days 00:00:01,2018-07-01
2,3,rent,condo,"dtdistrict15,dtdistrict16,dtdistrict18",2000.0,4000.0,web,u1386,0 days 00:03:37,2018-07-01
3,4,rent,condo,"dtdistrict15,dtdistrict16,dtdistrict18",2000.0,4000.0,web,u1386,0 days 00:00:01,2018-07-01
4,5,rent,condo,"dtdistrict15,dtdistrict16,dtdistrict18",2000.0,4000.0,web,u1386,0 days 00:02:40,2018-07-01


In [139]:
searches_agg_df = searches_df.groupby(['user_id', 'search_date','listing_type', 'property_type', 'search_location','search_price_min', 'search_price_max' ,'platform']).agg(
    total_duration=pd.NamedAgg(column='session_duration', aggfunc=sum),
    search_count=pd.NamedAgg(column='user_id', aggfunc=len)
)

searches_agg_df.head()


total_duration  search_count
user_id search_date listing_type property_type search_location           search_price_min search_price_max platform                              
u1      2018-07-13  rent         condo         dtdistrict05              1000.0           2000.0           web      0 days 00:00:47             3
                                                                                          2500.0           web      0 days 00:08:38             1
u1017   2018-07-19  sale         condo         dtdistrict15              700000.0         1000000.0        web      0 days 00:00:50             8
u1019   2018-07-18  rent         condo         dtdistrict15              1000.0           1600.0           web      0 days 00:00:33             2
u102    2018-07-31  sale         landed        dtdistrict10,dtdistrict11 2700000.0        3600000.0        web      0 days 00:00:29             2

In [146]:
enquiries_df = pd.read_csv(io.BytesIO(enquiries['99.co Data Analyst Challenge - enquiries.tsv']), sep='\t')
listings_df = pd.read_csv(io.BytesIO(listings['99.co Data Analyst Challenge - listings.tsv']), sep='\t')
users_df = pd.read_csv(io.BytesIO(users['99.co Data Analyst Challenge - users.tsv']), sep='\t')

# Merge dataframe Searches Agg & Listing
# merged_df = pd.merge(searches_agg_df, enquiries[['user_id','listing_type','property_type', 'search_price_min','search_price_max', 'platform', 'total_duration', 'search_count']], how='left', left_on='user_id', right_on='user_id').drop(columns = ['search_date']).fillna(0)


KeyError: ignored

## **Exploratory Data Analysis**

Considering important factors that affect characterization of a session search

1. Property type
2. Listing type
3. Property location
4. Search time
5. Range Price

## **Clustering**

## **Recommendation for Product Team**

The goal is to create more conversion, means to decrease drop rate in user funnel. It will be better if we have complete user journey data, e.g :
1. Total sessions
2. Total sessions with product view
3. Total sessions with enquiries
4. Total session with transaction

So we can strategy our different approach based on different goals, whether discovery, consideration & evaluation, and conversion.

In this section, I will share some insights and recommendation that related to conversion stage.
1. Around 87% users who searched on July 2018 didn't do enquiry. It means we can focus on those users, by giving them some kind of notification related to their preferences, e.g customized message for different cluster of users, giving similar property based on what they searched. Please check sheet ***-pivot search & enq***
2. 100% search on July 2018 happened on Web platform. Assuming the mobile application has been developed, there should be something wrong with marketing for moblie apps. Please check sheet ***-pivot searches, 3rd table***
3. 54% of our users searched for condo property, that is rented or sale. It means we should create more partnership with more developers who sell or rent condo. Second biggest percentage is sale HDB property. So it's clear that we should focus on rent & sale condo and sale HDB. Please check sheet ***-pivot searches, 1st table***
4. 27% of our users searched property for maximum price 2000,2500,3000 & 3500. Those prices are the biggest contributor in maximum price distribution. While the other max price are outlier numbers, on average 0%-2%. Please check sheet ***-pivot searches, 2nd table***